# Exercises

1. If you have trained 5 different on the exact same training data, & they all achieve 95% precision, is there any chance that you can combine these models to get better results? If so, how? If not, why?
2. What is the difference between hard & soft voting classifiers?
3. Is it possible to speed up training of a baggin ensemble by distributing it across multiple servers? What about pasting ensembles, boosting ensembles, random forests, or stacking ensembles?
4. What is the benefit of out-of-bag evaluation?
5. What makes extra-trees more random than regular random forests? How can this extra randomness help? Are extra-trees slower or faster than regular random forests?
6. If your AdaBoost ensemble underfits the training data, which hyperparameters should you tweak & how?
7. If your gradient boosting ensemble overfits the training set, should you increase or decrease the learning rate?
8. Load the MNIST data, & split it into a training set, a validation set, & a test set (e.g., use 50,000 instances for training, 10,000 for validation, & 10,000 for testing). Then train various classifiers, such as a random forest classifier, an extra-trees classifier, & a SVM classifier. Next, try to combine them into an ensemble that outperforms each individual classifier on the validation set, using soft or hard voting. Once you have found one, try it on the test set. How much better does it perform compared to the individual classifiers?
9. Run the individual classifers from the previous exercise to make predictions on the validation set, & create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image, & the target is the image's class. Train a classifier on this new training set. Congratualtions, you have just trained a blender, & together with the classifiers it forms a stacking ensemble! Now evaluate the ensemble on the test set. For each image in the test set, make predictions with all your classifiers, then feed the predictions to the blender to get the ensemble's predictions. how does it compare to the voting classifier you trained earlier?

---